In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_moons
from mlxtend.plotting import plot_decision_regions
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# Bagging and Pasting

In [2]:
X, y = make_moons(n_samples=500, noise=0.30, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [3]:
plt.scatter(X_train[:, 0], X_train[:, 1], c=y_train)
plt.show()

In [4]:
plt.hist(y_train)
plt.hist(y_test)
plt.show()

* Scikit-Learn oferuje prosty interfejs API do **bagging** i **pasting** . 

* Poniższy kod uczy zespół złożony z 500 drzewa decyzyjnego, z które zostało nauczonych na 100 elementach ze zbioru uczącego losowo pobranych z zestawu treningowego ze zwracaniem 

* Jest to przykład **bagging**, ale jeśli chcesz użyć **pasting** zamiast tego, po prostu ustaw **bootstrap=False**. 

* Parametr **n_jobs** mówi Scikit-Learn ile rdzeni procesora można użyć do treningu i prognoz (-1 mówi Scikit-Learn, aby używać wszystkich dostępnych rdzeni):

In [6]:
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(random_state=42), n_estimators=500,
    max_samples=100, bootstrap=True, n_jobs=-1, random_state=42)
bag_clf.fit(X_train, y_train)
y_pred = bag_clf.predict(X_test)

The BaggingClassifier automatycznie wykonuje miękkie głosowanie zamiast twardego, jeśli klasyfikator bazowy może oszacować prawdopodobieństwa klasy (tj. jeśli ma metodę **predict_proba()**), co ma miejsce w przypadku klasyfikatorów Decision Trees.

In [8]:
print(accuracy_score(y_test, y_pred))

In [9]:
tree_clf = DecisionTreeClassifier(random_state=42)
tree_clf.fit(X_train, y_train)
y_pred_tree = tree_clf.predict(X_test)
print(accuracy_score(y_test, y_pred_tree))

# Zad. 

Narysuj wynik dla

* BaggingClassifier
* DecisionTreeClassifier

In [10]:
fig, axes = plt.subplots(2, figsize=(10, 8))

for clf, ax in zip([bag_clf, tree_clf], axes.flatten()):
    plot_decision_regions(X, y, clf, legend=2, ax=ax)
    ax.set_title(clf.__class__.__name__)

# Out-of-Bag Evaluation

* W przypadku metody **bagging** niektóre przykłady mogą być próbkowane kilka razy dla dowolnego predyktora  podczas gdy inne mogą nie być próbkowane w ogóle. 

* Domyślnie BaggingClassifier pobiera próbki ze zwracaniem (**bootstrap = True**), gdzie $m$ jest rozmiarem zestawu treningowego. 

* Oznacza to, że tylko około 63% instancji szkoleniowych jest próbkowanych średnio dla każdego z predyktorów. 

* Pozostałe 37% instancji szkoleniowych, które nie są próbkowane, nazywane są instancjami **out-of-bag (oob)**. 

* Zauważ, że nie są one takie same dla wszystkich predyktorów. 

* Ponieważ predykator nigdy nie widzi instancji **out-of-bag (oob)** podczas treningu, klasyfikator może być oceniany na tych instancjach. 

* Możesz ocenić komitet klasyfikatorów uśredniając oceny każdego z predyktorów na **out-of-bag (oob)**. 

In [13]:
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), 
    n_estimators=500,
    bootstrap=True, 
    n_jobs=-1, 
    oob_score=True)

bag_clf.fit(X_train, y_train)
bag_clf.oob_score_

In [14]:
y_pred = bag_clf.predict(X_test)
accuracy_score(y_test, y_pred)

In [19]:
fig = plt.figure(figsize=(10, 8))
plot_decision_regions(X, y, bag_clf)
plt.show()